In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load Metro Stations Dataset
df = pd.read_csv("Dataset/bengaluru-metro-stations.csv")

# Function to convert time to minutes since midnight
def time_to_minutes(time_str):
    h, m = map(int, time_str.split(":"))
    return h * 60 + m

# Define peak hours (8:45-10:20 AM & 4:20-7:00 PM)
PEAK_HOURS = [(8 * 60 + 45, 10 * 60 + 20), (16 * 60 + 20, 19 * 60)]

# Generate training data
train_data = []
for _, row in df.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    
    # Generate demand data for different times of the day
    for minutes in range(0, 1440, 10):  # Every 10 minutes
        demand = any(start <= minutes <= end for start, end in PEAK_HOURS)  # 1 if peak hours, else 0
        train_data.append([lat, lon, minutes, demand])

# Create DataFrame
train_df = pd.DataFrame(train_data, columns=["Latitude", "Longitude", "Time_Minutes", "Demand"])

# Split Data
X = train_df[["Latitude", "Longitude", "Time_Minutes"]]
y = train_df["Demand"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate Model
y_pred = model.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Save the Model
joblib.dump(model, "demand_prediction_model.pkl")
print("Model saved as demand_prediction_model.pkl")


Model Accuracy: 0.9994490358126722
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      1499
        True       1.00      1.00      1.00       316

    accuracy                           1.00      1815
   macro avg       1.00      1.00      1.00      1815
weighted avg       1.00      1.00      1.00      1815

Model saved as demand_prediction_model.pkl


In [ ]:
# Load the saved model
model = joblib.load("demand_prediction_model.pkl")

# Example Test Input: Latitude, Longitude, and Time
test_input = pd.DataFrame({
    "Latitude": [12.9716],   # Example: Bangalore
    "Longitude": [77.5946],

    "Time_Minutes": [9 * 60]  # 9:00 AM in minutes
})

# Predict Demand
prediction = model.predict(test_input)

# Output Result
print("Predicted Demand:", "High" if prediction[0] == 1 else "Low")


Predicted Demand: High


In [4]:
# Load the saved model
model = joblib.load("demand_prediction_model.pkl")

# Example Test Input: Latitude, Longitude, and Time
test_input = pd.DataFrame({
    "Latitude": [12.9716],   # Example: Bangalore
    "Longitude": [77.5946],
    "Time_Minutes": [12 * 60]  # 9:00 AM in minutes
})

# Predict Demand
prediction = model.predict(test_input)

# Output Result
print("Predicted Demand:", "High" if prediction[0] == 1 else "Low")


Predicted Demand: Low
